<div class="row">
  <div class="col-sm-8">
     <center>
        <h1>
        IGR 204 - Visualisation<br>
        Mini-projet (Pr�noms en France)<br>
        PARTIE 2 - Evolution dans l'espace'
            <br>
            <br>
            <center><img src="https://s3-eu-west-1.amazonaws.com/assets.atout-on-line.com/images/ingenieur/Fiches_Ecoles/telecom_paristech.jpg" alt="drawing" width="170"/></center>
        <hr class="bg-light" />
            <i>Léa PAPILLON, Pierrick LEROY, Yann RODRIGUEZ, Ramzi KHALIFA, Alexandre LE BRIS<br></i>
            <i>MS IA - 21/22<br></i>
        </h1>
    </center>
  </div>
  </div>
</div>

In [ ]:
!pip install geopandas
!pip install altair==4.2

In [2]:
import geopandas as gpd
import altair as alt
import pandas as pd
import panel as pn

# Reading our names data

Now, let's read in our dataset.  The exported data is in CSV format, but with a `;` separator instead of commas.  The INSEE data collapses rare names or where department-level information has been elided (presumably to protect individuals with uncommon names or who were one of the only ones born with that name in a given year).  We'll strip those out.

In [30]:
df = pd.read_csv('data/dpt2020.csv', sep=';')
df.drop(df[df.preusuel == '_PRENOMS_RARES'].index, inplace=True)
df.drop(df[df.dpt == 'XX'].index, inplace=True)

df.annais = df.annais.astype(int)

df.sample(5)

,sexe,preusuel,annais,dpt,nombre
3510418,2,SANAA,2018,33,5
1059183,1,LUDOVIC,1994,48,5
363634,1,DAVID,1980,13,191
1471596,1,RUBEN,2009,974,11
1622487,1,URBAIN,1916,59,3


# Loading map data

Next, let's load some map data of regions in France using `geopandas`.  These map data come from the [INSEE] and [IGN] and were processed into the `geojson` format we'll need to work with by [Gr�goire David].  Here's the [github] repository.

In this example, we'll work with the simplified departments tiles for the Hexagon, but that repository contains higher-resolution versions, the DOM-TOM, and more.

[Gr�goire David]: https://gregoiredavid.fr
[INSEE]: http://www.insee.fr/fr/methodes/nomenclatures/cog/telechargement.asp
[IGN]: https://geoservices.ign.fr/adminexpress
[github]: https://github.com/gregoiredavid/france-geojson/

In [31]:
depts = gpd.read_file('data/departements-version-simplifiee.geojson')

depts.sample(5)

,code,nom,geometry
36,36,Indre,"POLYGON ((1.32667 47.18623, 1.40143 47.21245, ..."
73,73,Savoie,"POLYGON ((6.80252 45.77837, 6.80842 45.72515, ..."
41,41,Loir-et-Cher,"POLYGON ((0.84122 48.10306, 0.87589 48.10944, ..."
2,03,Allier,"POLYGON ((3.03207 46.79491, 3.04907 46.75808, ..."
86,86,Vienne,"POLYGON ((-0.10212 47.06480, -0.09806 47.09135..."


Notice how `depts` is a geopandas dataframe.  We'll use it just as a regular `pandas` dataframe, but it includes the geometry info we need to be able to draw those regions when we pass them into Altair.  We just need to make sure that when we work with our data, we keep them in a geopandas dataframe and not a plain dataframe if we want to draw the departments.

In the next cell, notice how we do a right-merge to bring in department data into names.  We do this as a merge on `depts` because we need a geopandas dataframe.  Remember, `depts` is a geopandas dataframe, while `names` is a regular dataframe.  If we did a left merge on `names`, we'd end up with a regular pandas dataframe. After this merge, both `names` and `depts` will be geopandas dataframes.

**Hint:** Be careful when you do your data joins here.  It's easy to accidentally merge the wrong way to accidentally create a _much bigger_ dataset.

In [12]:
# Keep a reference around to the plain pandas dataframe, without geometry data, just in case
just_names = df
df = depts.merge(df, how='right', left_on='code', right_on='dpt')
df.sample(5)

,code,nom,geometry,sexe,preusuel,annais,dpt,nombre
777474,42,Loire,"POLYGON ((3.89953 46.27591, 3.90940 46.25773, ...",1,JEAN-CLAUDE,1980,42,3
1286273,60,Oise,"POLYGON ((1.78384 49.75831, 1.80898 49.75433, ...",1,PASCAL,1950,60,16
3017981,75,Paris,"POLYGON ((2.41634 48.84924, 2.46226 48.84254, ...",2,MARIE-C�CILE,1983,75,6
2463307,26,Dr�me,"POLYGON ((4.80049 45.29836, 4.85880 45.30895, ...",2,GEORGETTE,1904,26,10
85665,74,Haute-Savoie,"POLYGON ((6.80252 45.77837, 6.75551 45.76635, ...",1,ALLAN,2004,74,6


# Show a name over all years

Now we'll choose a name to show across all years.  To that, we'll group all of the names in a department together (squashing the years together) and use the sum.

In [6]:
grouped = df.groupby(['dpt', 'preusuel', 'sexe'], as_index=False).sum()
grouped = depts.merge(grouped, how='right', left_on='code', right_on='dpt') # Add geometry data back in
grouped

,code,nom,geometry,dpt,preusuel,sexe,nombre
0,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,AARON,1,160
1,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABBY,2,3
2,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABDALLAH,1,7
3,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABDEL,1,3
4,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABDELKADER,1,3
...,...,...,...,...,...,...,...
239574,NaN,NaN,None,974,�SA�E,1,3
239575,NaN,NaN,None,974,�THAN,1,53
239576,NaN,NaN,None,974,�TIENNE,1,3
239577,NaN,NaN,None,974,�VA,2,32


In [7]:
grouped['ratio'] = grouped.nombre/grouped.groupby('dpt').nombre.transform('sum')

Now let's pick a name and check out how it's distribution over the last 120 years across Metropolitan France.  In this example, I choose the name �Lucien,� which I rather like for some reason.

In [29]:
def plot_loc(df_selection):
    chart = alt.Chart(df_selection).mark_geoshape(stroke='white').encode(
                tooltip=['nom', 'code', 'nombre'],
                color='ratio',
            ).properties(width=800, height=600)
    
    return chart

In [27]:
def select_data_loc(annee_range, prenom_regex='paul'):
    
    filter_annee = ((df.annais >= int(annee_range[0])) &
                    (df.annais <= int(annee_range[1])))
    
    df_selection = df[filter_annee]
    
    df_selection = df_selection.groupby(['dpt', 'preusuel', 'sexe'], as_index=False).sum()
    df_selection = depts.merge(df_selection, how='right', left_on='code', right_on='dpt') 

    df_selection['ratio'] = df_selection.nombre/df_selection.groupby('dpt').nombre.transform('sum')

    filter_prenom = df_selection.preusuel.str.contains(prenom_regex.upper(), regex=True, na=False)
    df_selection = df_selection[filter_prenom]

    return plot_loc(df_selection)

In [28]:
text = ("## Discover name trends by location !\n")

name = pn.widgets.TextInput(name='Regex Pr�nom (ajouter $ pour prenom uniquement)',
                            value='paul$',
                            placeholder='Taper la requ�te (regex possible)')

annee_range = pn.widgets.IntRangeSlider(name='Ann�e(s)',
                                        start=1900, end=2020,
                                        value=(1900, 2020),
                                        step=1)

widget = pn.Column(name,
                   annee_range)

reactive_selection = pn.bind(select_data_loc,
                            annee_range,
                            name)

plot = pn.Column(text,
                 widget,
                 reactive_selection)

plot.show();

Launching server at http://localhost:57535


-----------------------------

# Plotly + Dash version

## Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output, callback_context

## Reading data

In [2]:
names = pd.read_csv("dpt2020.csv", sep=";")
names.drop(names[names.preusuel == '_PRENOMS_RARES'].index, inplace=True)
names.drop(names[names.dpt == 'XX'].index, inplace=True)

depts = gpd.read_file('departements-version-simplifiee.geojson')

names = depts.merge(names, how='left', left_on='code', right_on='dpt')

names = names.dropna()
names["annais"] = pd.to_numeric(names["annais"])
names["preusuel"] = names["preusuel"].astype("string")

names

,code,nom,geometry,sexe,preusuel,annais,dpt,nombre
0,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",1.0,AARON,2005,01,3.0
1,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",1.0,AARON,2007,01,4.0
2,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",1.0,AARON,2008,01,6.0
3,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",1.0,AARON,2009,01,7.0
4,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",1.0,AARON,2010,01,8.0
...,...,...,...,...,...,...,...,...
3446640,95,Val-d'Oise,"POLYGON ((2.59052 49.07965, 2.57203 49.06149, ...",2.0,ZOHRA,2020,95,4.0
3446641,95,Val-d'Oise,"POLYGON ((2.59052 49.07965, 2.57203 49.06149, ...",2.0,ZORA,1978,95,3.0
3446642,95,Val-d'Oise,"POLYGON ((2.59052 49.07965, 2.57203 49.06149, ...",2.0,ZOUBIDA,1984,95,3.0
3446643,95,Val-d'Oise,"POLYGON ((2.59052 49.07965, 2.57203 49.06149, ...",2.0,ZOYA,2013,95,3.0


## Building graph

In [5]:
app = JupyterDash()

app.layout = html.Div([

    html.Div([
        dcc.Graph(id='our_graph')
    ],className='nine columns'),

    html.Div([
        html.Br(),
        html.Div(id='output_data'),
        html.Br(),
        html.Label(['Pr�nom:'],style={'font-weight': 'bold', "text-align": "center"}),

        # Search bar
        dcc.Input(id='my_searchbar', type="text", placeholder="", debounce=True),
        # "OK" button
        html.Button('OK', id='submit-val', n_clicks=0),
        html.Div(id='container-button-basic',
                children='Sexe : '),
        # Male, Female, M+F buttons
        html.Button('F', id='btn-nclicks-1', n_clicks=0),
        html.Button('M', id='btn-nclicks-2', n_clicks=0),
        html.Button('F+M', id='btn-nclicks-3', n_clicks=0),
        html.Div(id='container-button-timestamp')
    ],className='three columns'),

])

@app.callback(
    Output(component_id='our_graph', component_property='figure'),
    [Input(component_id='my_searchbar', component_property='value'),
    Input('btn-nclicks-1', 'n_clicks'),
    Input('btn-nclicks-2', 'n_clicks'),
    Input('btn-nclicks-3', 'n_clicks')]
)

def build_graph(column_chosen, btn1, btn2, btn3):
    dff = names.copy()

    if column_chosen is not None :
        dff = dff[dff["preusuel"] == column_chosen.upper()]
        print(column_chosen.upper())

    # Changing gender
    changed_id = [p['prop_id'] for p in callback_context.triggered][0]
    if 'btn-nclicks-1' in changed_id:
        dff = dff[dff["sexe"] == 2]
        print("F")
    elif 'btn-nclicks-2' in changed_id:
        dff = dff[dff["sexe"] == 1]
        print("H")
    elif 'btn-nclicks-3' in changed_id:
        dff = dff
        print("F+H")
    
    # Choropleth map
    fig=px.choropleth(dff,
                geojson=depts,
                featureidkey='properties.code',
                locations='code',
                animation_frame='annais',
                # color='ratio',
                color='nombre',
                color_continuous_scale='Inferno',
                hover_data=['annais'],
                projection="mercator",
                title='Births',
                # range_color=[0, 6400]
                height=1000,
                width=1000
                )
    fig.update_geos(fitbounds="locations", visible=True)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig

In [6]:
# NOTE: JupyterDash allows to visualize directly in a Jupyter Notebook. However, it could be better to visualize in a web browser.
app.run_server(host='127.0.0.1', port=8040, mode='inline', debug=False)

 * Running on http://127.0.0.1:8040/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jun/2022 16:18:32] "GET /_alive_014bdc58-dd9c-4d1f-854b-579f31dae980 HTTP/1.1" 200 -


127.0.0.1 - - [17/Jun/2022 16:18:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2022 16:18:32] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2022 16:18:32] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2022 16:18:32] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2022 16:18:32] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


CAMILLE


127.0.0.1 - - [17/Jun/2022 16:19:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jun/2022 16:19:50] "POST /_dash-update-component HTTP/1.1" 200 -
